In [5]:
import pandas as pd
import numpy as np
import glob
import re
import networkx as nx
from collections import defaultdict
import itertools
import matplotlib.pyplot as plt
import json
import requests
import gzip
from bs4 import BeautifulSoup
import re
import os

%load_ext autoreload
%autoreload 2
import lib.build_graph as bg
import lib.create_database as cdb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/jeffliao/Desktop/dsc180a/project/lib/create_database.py:19: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 19 of the file /Users/jeffliao/Desktop/dsc180a/project/lib/create_database.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(site.text)


In [3]:
# get files and data
# cfg = json.load(open('config/n_app.json'))
# db.main(**cfg)

In [96]:
def main(smali_src, number_of_smali, **kwargs):
    # initalize data
    smali = cdb.get_smali(smali_src)
    test = smali[:number_of_smali]
    
#     calls = list()

    for f in test:
        blocks = defaultdict(list)
        same_block = defaultdict(list)
        
        with open(f) as file:
            # record all api calls
#             bg.find_api_calls(file, calls)
#             file.seek(0)
            
            # build B
            bg.locate_method_blocks(file, blocks)
            for b in blocks:
                apis = []
                bg.find_api_calls(blocks[b], apis)
                same_block[b] = apis
            bg.create_edges(same_block, B_graph)
            
            file.seek(0)
            # build P
            for line in file:
                if('invoke-' in line and 'method' not in line):
                    package = bg.find_package(line)
                    method = bg.find_method(line)
                    packageD[package].append(method)
                    
                    api = []
                    bg.find_api_calls([line], api)
                    invoke = bg.find_invoke(line)
                    invokeD[invoke].append(api[0])

In [97]:
%%time

B_graph = nx.Graph()
P_graph = nx.Graph()
I_graph = nx.Graph()

packageD = defaultdict(list)
invokeD = {'direct': [], 'virtual':[], 'static':[], 'super':[],'interface':[]}

cfg = json.load(open('config/1_app.json'))
main(**cfg)
bg.create_edges(packageD, P_graph)
bg.create_edges(invokeD, I_graph)
A, apis, apks = bg.buildA_matrix('./data/smali')

game_arcade-84
game_trivia-33
tools-115
personalization-1
tools-60
tools-381
game_simulation-4
business-15
education-117
game_casual-183
travel_and_local-102
travel_and_local-104
entertainment-461
lifestyle-133
news_and_magazines-41
news_and_magazines-23
tools-276
shopping-56
medical-46
personalization-102
entertainment-334
entertainment-536
medical-59
social-82
auto_and_vehicles-6
entertainment-313
tools-30
game_puzzle-162
game_puzzle-165
lifestyle-282
travel_and_local-108
news_and_magazines-113
entertainment-81
productivity-12
entertainment-19
lifestyle-75
game_action-89
productivity-13
game_action-26
lifestyle-74
music_and_audio-94
entertainment-274
music_and_audio-18
entertainment-93
lifestyle-50
shopping-105
video_players-11
game_casual-54
game_arcade-34
business-433
CPU times: user 47.5 s, sys: 14.6 s, total: 1min 2s
Wall time: 1min 28s


In [98]:
B = nx.adjacency_matrix(B_graph, apis.values)
I = nx.adjacency_matrix(I_graph, apis.values)

In [125]:
nx.adjacency_matrix(P_graph)

<95x95 sparse matrix of type '<class 'numpy.int64'>'
	with 1328 stored elements in Compressed Sparse Row format>

In [26]:
ABA = A*B*A.transpose()

In [24]:
A.shape

(50, 60503)

In [25]:
B.shape

(60503, 60503)

In [27]:
ABA.shape

(50, 50)

In [28]:
ABA

matrix([[65, 52, 52, ..., 51, 31, 65],
        [52, 45, 45, ..., 45, 28, 52],
        [52, 45, 45, ..., 45, 28, 52],
        ...,
        [51, 45, 45, ..., 45, 28, 51],
        [31, 28, 28, ..., 28, 18, 31],
        [65, 52, 52, ..., 51, 31, 65]], dtype=int64)